In [5]:
from Cryptodome.Random import get_random_bytes
from Cryptodome.Hash import SHA256
from Cryptodome.PublicKey import RSA
from Cryptodome.Cipher import PKCS1_OAEP

import base64

In [6]:
def b64s(bytes):
    return base64.b64encode(bytes).decode("utf-8")

In [3]:
def gen_salt(size=18):
    return get_random_bytes(size)

In [7]:
b64s(gen_salt(18))

'FIuVblUglg+BAJGMKr61EwbA'

In [11]:
# Client generates password hash and returns it to the server

def client_generate(pubsalt, pubkey, password):
    # salt
    salted = bytes(password, 'utf-8') + pubsalt
    # hash
    hash = SHA256.new()
    hash.update(salted)
    digest = hash.digest()
    # encrypt
    encryptor = PKCS1_OAEP.new(pubkey)
    encrypted = encryptor.encrypt(digest)
    # send
    return encrypted

In [13]:
# Server registration simulation

def server_register(password):
    # generate context
    pubsalt = gen_salt(18)
    privsalt = gen_salt(18)    
    rsa = RSA.generate(1024)    
    # wait for client
    user_hash = client_generate(pubsalt, rsa.public_key(), password)
    # decrypt
    decryptor = PKCS1_OAEP.new(rsa)
    decrypted = decryptor.decrypt(user_hash)
    # hash
    hash = SHA256.new()
    hash.update(decrypted + privsalt)
    final = base64.b64encode(hash.digest())
    # store
    return (pubsalt, privsalt, final)

In [17]:
user = server_register('password2')
( b64s(user[0]), b64s(user[1]), b64s(user[2]) )

('NGMO3fskSCxl6+S4b2LLVq+r',
 'Qbyy3Q2Lui4Ns+jKxfLUL4BN',
 'V2kwbW9SK1RURVNxMUNHVWMxRUg0bXdjd0tBOHRsaVlweklhMnp0eXF6Zz0=')

In [20]:
# Server login simulation

def server_login(user, password):
    # generate context
    pubsalt = user[0]
    privsalt = user[1]
    rsa = RSA.generate(2048)    
    # wait for client
    user_hash = client_generate(pubsalt, rsa.public_key(), password)
    # decrypt    
    decryptor = PKCS1_OAEP.new(rsa)
    decrypted = decryptor.decrypt(user_hash)
    # hash
    hash = SHA256.new()
    hash.update(decrypted + privsalt)
    final = base64.b64encode(hash.digest())
    # verify
    return user[2] == final

In [21]:
server_login(user, 'password2')

True